**Convolutional Net Categorical Image Classifier**

Basic CNN for classifying a set of image doodles

Setup function is below

In [14]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import CSVLogger

GOOGLE_COLAB = True
TRAINING_LOGS_FILE = "/content/drive/My Drive/Colab Notebooks/"+"training_logs.csv"
MODEL_SUMMARY_FILE = "/content/drive/My Drive/Colab Notebooks/"+"model_summary.txt"
TEST_FILE = "/content/drive/My Drive/Colab Notebooks/"+"test_file.txt"
MODEL_FILE = "/content/drive/My Drive/Colab Notebooks/"+"model.h5"

!pip install livelossplot
from livelossplot import PlotLossesKeras

**For use with Google Cloud**



In [0]:
#from google.cloud import storage
#client = storage.Client()
#bucket = client.get_bucket('bucket-id-here')

**OR using Drive storage**

In [15]:
from google.colab import drive, files
drive.mount('/content/drive/')
path = "/content/drive/My Drive/Colab Notebooks/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Loading the data

In [0]:
initial_labels = ['aircraft']

labels = ['banana', 'bear', 'bird', 'cat', 'cloud', 'elephant', 'face', 'golfclub', 'hotdog']

#f_path = path + "data/"+p+".npy"
#banana_path = path + "data/banana.npy"
#bear_path = path + "data/bear.npy"

#aircraft_loaded = np.load(aircraft_path)
#banana_loaded = np.load(banana_path)
#bear_loaded = np.load(bear_path)

f_path = path + "data/"+initial_labels[0]+".npy"
f_loaded = np.load(f_path)
y = np.zeros(f_loaded.shape[0])

train_X, test_X, train_y, test_y = train_test_split(f_loaded, y, test_size=0.1, random_state=42)

for i, label in enumerate(labels):
  f_path = path + "data/"+label+".npy"
  f_loaded = np.load(f_path)
  
  
  num_ex = f_loaded.shape[0]
  
  f_split = f_loaded[:int(num_ex/10)] 
  
  y = np.ones(f_split.shape[0])*(i+1)
  
  X_train, X_test, y_train, y_test = train_test_split(f_split, y, test_size=0.1, random_state=42)
  
  train_X = np.concatenate((train_X, X_train), axis=0)
  test_X = np.concatenate((test_X, X_test), axis=0)
  train_y = np.concatenate((train_y, y_train), axis=0)
  test_y = np.concatenate((test_y, y_test), axis=0)


Run this code to save the data

In [17]:
print('Train Size: '+ str(train_X.shape[0]))
print('Train Label Size: '+ str(train_y.shape[0]))
print('Test Size: '+ str(test_X.shape[0]))
print('Test Label Size: '+ str(test_y.shape[0]))
np.save(path+'data/train_x.npy', train_X)
np.save(path+'data/train_y.npy', train_y)
np.save(path+'data/test_x.npy', test_X)
np.save(path+'data/test_y.npy', test_y)

Train Size: 238515
Train Label Size: 238515
Test Size: 26506
Test Label Size: 26506


In [18]:
print(str(train_X[0].shape[0]))
print(train_X.shape)
print(train_X[0])

784
(238515, 784)
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0 

Hyper Params

In [0]:
IMAGE_SIZE = 28
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 3
BATCH_SIZE = 32
TEST_SIZE = 30

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 1)


Model

In [0]:
model = Sequential()

model.add(Conv2D(9, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(9, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(18, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(18, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(36, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(36, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(72, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(72, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(72, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(72, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10))
model.add(Activation('sigmoid'))
    
model.compile(loss='categorical_crossentropy',
            optimizer=RMSprop(lr=0.0001),
            metrics=['accuracy'])

with open(MODEL_SUMMARY_FILE,"w") as fh:
    model.summary(print_fn=lambda line: fh.write(line + "\n"))



Execution

In [21]:
# Training
# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(train_y, num_classes=10)

num_train = train_X.shape[0]

X_train = train_X.reshape((num_train, 28, 28, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(X_train, one_hot_labels, epochs=EPOCHS, batch_size=BATCH_SIZE)
model.save_weights(MODEL_FILE)

#if GOOGLE_COLAB:
#    files.download(MODEL_SUMMARY_FILE)
#    files.download(MODEL_FILE)
#    files.download(TRAINING_LOGS_FILE)
#    files.download(TEST_FILE)

Epoch 1/3
238515/238515 [==============================] - 106s 443us/step - loss: 1.2028 - acc: 0.6102
Epoch 2/3
238515/238515 [==============================] - 105s 442us/step - loss: 0.7889 - acc: 0.7571
Epoch 3/3
238515/238515 [==============================] - 105s 439us/step - loss: 0.6958 - acc: 0.7972
